In [ ]:
# !pip install PIMS
# !pip install trackpy
# !pip install pandas==1.5.3
# !pip install opencv-python
# !pip install plotly
# !pip install "numpy<2"
# !pip install stardist
# !pip install tensorflow

In [ ]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

import numpy as np
print(np.__version__)
import pandas as pd

import pims
import trackpy as tp
# print(tp.__version__)
import os

import cv2

import matplotlib  as mpl 
import matplotlib.pyplot as plt 
import plotly.express as px

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(15, 12)) #(10, 6)
mpl.rc('image', cmap='gray')

In [ ]:
# Parameters

sample = 1
series = 0
subarea = 1

In [ ]:
# to read standard files, including tiff stacks, we can use pims.open
my_img = pims.open(f'raw_data/sample_{sample}/tiff_series_{sample}_{series}/{subarea}/*.tif')

In [ ]:
my_img

In [ ]:
my_img.shape

In [ ]:
from stardist.models import StarDist2D, Config2D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

In [ ]:
# prints a list of available models
StarDist2D.from_pretrained()

In [ ]:
# define a pretrained model to segment nuclei in fluorescence images (download from pretrained)
model = StarDist2D.from_pretrained('2D_versatile_fluo') # 2D_versatile_he

In [ ]:
plt.figure(figsize=(9, 6))
plt.imshow(my_img[0], cmap='gray')
plt.axis('off')

In [ ]:
@pims.pipeline
def stardist_segm(img):
#     img = cv2.resize(img, None, fx = 2, fy = 2)
    img_labels, img_details = model.predict_instances(normalize(img, 1.0, 99.8), prob_thresh=0.7, nms_thresh=0.8)
    return img_labels

In [ ]:
label_image = stardist_segm(my_img)

In [ ]:
label_image

In [ ]:
img_num = 1

plt.subplot(1, 2, 1)
plt.imshow(my_img[img_num], cmap='gray')
plt.axis('off')
plt.title('input image')

plt.subplot(1, 2, 2)
plt.imshow(render_label(label_image[img_num], img=my_img[img_num]))
# plt.imshow(he_labels)
plt.axis('off')
plt.title('prediction + input overlay')

In [ ]:
import skimage

In [ ]:
%%time
features = pd.DataFrame()

for num, img in enumerate(my_img):
    for region in skimage.measure.regionprops(label_image[num], intensity_image=img):
        # store fuatures
        dataset = pd.DataFrame({
            'y': [region.centroid[0]],
            'x': [region.centroid[1]],
            'frame': [num],
            'area': [region.area],
            'brightness': [region.intensity_mean],
        })
        
        features = pd.concat([features, dataset])

In [ ]:
features.to_csv(f'data_out/sample_{sample}/series_{series}/{subarea}/heatmap_{sample}_{series}.csv.zip', compression='gzip', index=False)

In [ ]:
# read 'features' from file 

# features = pd.read_csv('data_out/S6T34to18_4_0_2000.csv.zip', compression='gzip')
# features.head

In [ ]:
# create copy of features to make modifications
features_modif = features.copy()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(9)], my_img[9], plot_style={'markersize':2});


In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], nbins=100, template="simple_white")

fig.show()

In [ ]:
# filter out by intensity, then give it the name feature as it was originally

features_modif = features_modif.loc[(features_modif['brightness'] > 160)]

In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], nbins=20, template="simple_white")

fig.show()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(5)], my_img[5], plot_style={'markersize':2});

In [ ]:
# Bubble tracking
# we must specify a maximum displacement, the farthest an object can travel between frames (search_range)
# We allow for the possibility that an object might be missed for a few frames and then
# Memory keeps track of disappeared objects and maintains their ID for up to some number
# Here we use 5 frames.
# Note that the term particle refers to an object of interest.

search_range = 5 #4 #3 #10 #1 
t = tp.link_df(features_modif, search_range, memory=5) # memory=5
t.head

In [ ]:
# is there an overall drift? If so, we need to correct for it
drift = tp.compute_drift(t)

In [ ]:
t.head()

In [ ]:
drift.plot(figsize=(10, 6)) # if there is a horizontal line, there is no drift
plt.show()

In [ ]:
# correct drift
t_corrected = tp.subtract_drift(t.copy(), drift)
t_corrected.reset_index(drop=True, inplace=True)

In [ ]:
# plot drift plots one more to be sure it works well
drift = tp.compute_drift(t_corrected)

drift.plot(figsize=(10, 6)) # if there is a horizontal line, there is no drift
plt.show()

In [ ]:
t_corrected.to_csv(f'data_out/sample_{sample}/series_{series}/{subarea}/heatmap_{sample}_{series}.csv', index=False)

In [ ]:
# Ensemble Mean Squared Displacement

# !!! This doesn't work for the latest pandas version (2.0.3). Downgrade version to 1.5.3 (pip install pandas==1.5.3)

em = tp.emsd(t_corrected, 0.06905, 60., max_lagtime=1000)

In [ ]:
sqrt_em = np.sqrt(em)

In [ ]:
lagt_crit = 1 #sqrt_em.index[-1]
max_sqrt_em = sqrt_em[sqrt_em.index >= lagt_crit].values[0]
max_sqrt_em

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(im.index, im, 'o', color='red')
ax.plot(em.index, sqrt_em, 'o')
# tp.utils.fit_powerlaw(sqrt_em)
ax.axvline(x=lagt_crit, color='green')
ax.axhline(y=max_sqrt_em, color='green')
ax.set(ylabel=r'$\sqrt{\langle \Delta r^2 \rangle}$ [$\mu$m]',
      xlabel='lag time $t$')
ax.set_xlim([0.01, 200])
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
a_s = 1.28 # micrometers

R = np.sqrt(3/4) * max_sqrt_em
R

In [ ]:
# write R in file
import os.path
import csv

fname = f"data_out/sample_{sample}/R_{sample}.csv"
if os.path.exists(fname) == False:
    with open(fname, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["series", "subarea", "R"])

with open(fname, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([series, subarea, R])